# LASSO vs Baseline: Out-of-Sample Comparison

Compares two models for predicting ordinary stock returns:

1. **Baseline**: `predicted_return = beta * index_return`
2. **LASSO**: `predicted_return = beta * index_return + LASSO(russell_residuals)`

All comparisons use truly out-of-sample test periods (walk-forward validation).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

%matplotlib inline

## 1. Load Data

In [ ]:
# Out-of-sample evaluation results
results = pd.read_csv('../data/processed/models/with_us_stocks/lasso_evaluation/baseline_comparison.csv')
results['month'] = pd.to_datetime(results['month'])

# Training metadata (for model details)
metadata = pd.read_csv('../data/processed/models/with_us_stocks/lasso/training_metadata.csv')
metadata['model_date'] = pd.to_datetime(metadata['model_date'])

print(f"Evaluation results: {len(results)} model-months, {results['ticker'].nunique()} stocks")
print(f"Date range: {results['month'].min():%Y-%m} to {results['month'].max():%Y-%m}")
results.head()

## 2. Overall Comparison

In [ ]:
print(f"Mean Baseline IC: {results['baseline_ic'].mean():.4f}")
print(f"Mean LASSO IC:    {results['lasso_ic'].mean():.4f}")
print(f"Mean Improvement: {results['ic_improvement'].mean():+.4f}")
print(f"Median Improvement: {results['ic_improvement'].median():+.4f}")
print(f"% months LASSO wins: {(results['ic_improvement'] > 0).mean() * 100:.1f}%")

# Paired t-test
from scipy import stats
t_stat, p_val = stats.ttest_rel(results['lasso_ic'], results['baseline_ic'])
print(f"\nPaired t-test: t={t_stat:.3f}, p={p_val:.4f}")

## 3. Per-Stock Comparison

In [ ]:
# Per-stock averages
stock_comparison = results.groupby('ticker').agg(
    baseline_ic=('baseline_ic', 'mean'),
    lasso_ic=('lasso_ic', 'mean'),
    improvement=('ic_improvement', 'mean'),
    pct_lasso_wins=('ic_improvement', lambda x: (x > 0).mean() * 100),
    n_months=('ic_improvement', 'count')
).sort_values('improvement', ascending=False).round(4)

stock_comparison

In [ ]:
# Bar chart: IC improvement by stock
fig, ax = plt.subplots(figsize=(14, 7))
colors = ['tab:green' if v > 0 else 'tab:red' for v in stock_comparison['improvement']]
ax.barh(stock_comparison.index, stock_comparison['improvement'], color=colors, edgecolor='black', linewidth=0.5)
ax.axvline(0, color='black', linewidth=1)
ax.set_xlabel('IC Improvement (LASSO - Baseline)')
ax.set_title('Out-of-Sample IC Improvement by Stock')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

print(f"Stocks where LASSO improves: {(stock_comparison['improvement'] > 0).sum()} / {len(stock_comparison)}")

## 4. Scatter: Baseline IC vs LASSO IC

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(stock_comparison['baseline_ic'], stock_comparison['lasso_ic'], s=60, alpha=0.7)

# Label each point
for ticker, row in stock_comparison.iterrows():
    ax.annotate(ticker, (row['baseline_ic'], row['lasso_ic']),
                fontsize=7, ha='left', va='bottom')

# 45-degree line
lims = [min(ax.get_xlim()[0], ax.get_ylim()[0]), max(ax.get_xlim()[1], ax.get_ylim()[1])]
ax.plot(lims, lims, 'k--', alpha=0.5, label='LASSO = Baseline')
ax.set_xlim(lims); ax.set_ylim(lims)
ax.set_xlabel('Baseline IC (mean over months)')
ax.set_ylabel('LASSO IC (mean over months)')
ax.set_title('Per-Stock: LASSO IC vs Baseline IC')
ax.set_aspect('equal')
ax.legend()
plt.tight_layout()
plt.show()

## 5. IC Improvement Over Time

In [ ]:
# Average across stocks per month
monthly = results.groupby('month').agg(
    baseline_ic=('baseline_ic', 'mean'),
    lasso_ic=('lasso_ic', 'mean'),
    improvement=('ic_improvement', 'mean'),
    pct_lasso_wins=('ic_improvement', lambda x: (x > 0).mean() * 100)
)

fig, axes = plt.subplots(2, 1, figsize=(14, 9), sharex=True)

# Top: Baseline vs LASSO IC over time
axes[0].plot(monthly.index, monthly['baseline_ic'], 'o-', label='Baseline', linewidth=2)
axes[0].plot(monthly.index, monthly['lasso_ic'], 's-', label='LASSO', linewidth=2)
axes[0].axhline(0, color='black', linewidth=0.5)
axes[0].set_ylabel('Mean IC')
axes[0].set_title('Mean IC Over Time (averaged across stocks)')
axes[0].legend()

# Bottom: IC improvement over time
colors = ['tab:green' if v > 0 else 'tab:red' for v in monthly['improvement']]
axes[1].bar(monthly.index, monthly['improvement'], width=20, color=colors, edgecolor='black', linewidth=0.5)
axes[1].axhline(0, color='black', linewidth=1)
axes[1].set_ylabel('Mean IC Improvement')
axes[1].set_xlabel('Month')
axes[1].set_title('LASSO IC Improvement Over Time')

plt.tight_layout()
plt.show()

## 6. Heatmap: IC Improvement by Stock and Month

In [ ]:
improvement_pivot = results.pivot(index='ticker', columns='month', values='ic_improvement')
# Sort by mean improvement
improvement_pivot = improvement_pivot.loc[stock_comparison.index]

fig, ax = plt.subplots(figsize=(16, max(8, len(improvement_pivot) * 0.35)))
sns.heatmap(improvement_pivot, cmap='RdYlGn', center=0, vmin=-0.3, vmax=0.3,
            cbar_kws={'label': 'IC Improvement'}, ax=ax)
ax.set_title('IC Improvement Heatmap (LASSO - Baseline)')
ax.set_xlabel('Month')
ax.set_ylabel('Stock')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 7. Distribution of IC Improvement

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Per model-month
axes[0].hist(results['ic_improvement'], bins=40, edgecolor='black', alpha=0.7)
axes[0].axvline(0, color='red', linewidth=2, linestyle='--')
axes[0].axvline(results['ic_improvement'].mean(), color='green', linewidth=2,
                linestyle='--', label=f"Mean = {results['ic_improvement'].mean():+.3f}")
axes[0].set_xlabel('IC Improvement')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of IC Improvement (all model-months)')
axes[0].legend()

# Per stock (averaged)
axes[1].hist(stock_comparison['improvement'], bins=20, edgecolor='black', alpha=0.7)
axes[1].axvline(0, color='red', linewidth=2, linestyle='--')
axes[1].axvline(stock_comparison['improvement'].mean(), color='green', linewidth=2,
                linestyle='--', label=f"Mean = {stock_comparison['improvement'].mean():+.3f}")
axes[1].set_xlabel('Mean IC Improvement')
axes[1].set_ylabel('Number of Stocks')
axes[1].set_title('Distribution of Mean IC Improvement (per stock)')
axes[1].legend()

plt.tight_layout()
plt.show()

## 8. Model Characteristics (Sparsity and Alpha)

In [ ]:
# Merge metadata with results to see if sparsity/alpha relate to improvement
merged = results.merge(
    metadata[['ticker', 'model_date', 'alpha', 'n_nonzero_coefs', 'val_ic']],
    left_on=['ticker', 'month'], right_on=['ticker', 'model_date'], how='left'
)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Sparsity vs improvement
axes[0].scatter(merged['n_nonzero_coefs'], merged['ic_improvement'], alpha=0.3, s=20)
axes[0].axhline(0, color='red', linewidth=1, linestyle='--')
axes[0].set_xlabel('Number of Non-Zero Coefficients')
axes[0].set_ylabel('IC Improvement')
axes[0].set_title('Model Sparsity vs IC Improvement')

# Alpha vs improvement
axes[1].scatter(np.log10(merged['alpha']), merged['ic_improvement'], alpha=0.3, s=20)
axes[1].axhline(0, color='red', linewidth=1, linestyle='--')
axes[1].set_xlabel('log10(Alpha)')
axes[1].set_ylabel('IC Improvement')
axes[1].set_title('Regularization Strength vs IC Improvement')

plt.tight_layout()
plt.show()

print(f"Correlation(n_nonzero_coefs, improvement): {merged['n_nonzero_coefs'].corr(merged['ic_improvement']):.3f}")
print(f"Correlation(log_alpha, improvement): {np.log10(merged['alpha']).corr(merged['ic_improvement']):.3f}")